# Action plan

## Goal

The goal here is to take in a very large point cloud and use mini batch k means to cluster the points into texture/land types. 

To do this, I must be able to read in the data in batches.

The data I want to read in are points with associated patches about them that have features for composed of their neighbours information. 

e.g. 

for a point at (lon, lat) = (x0, y0), I take it and its closest 8 neighbours { (x1, y1), ..., (x8, y8) } and use them to poll the data to construct the feature vector/array ((z0, r0, g0, b0), (z1, r1, g1, b1), ..., (z8, r8, g8, b8)). 

If I could get this data for 1000 points at a time, then I could do 1000-pixel minibatches of minibatchKMeans. 

# Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from DataLoader import DataLoader

In [ ]:
from helper_functions import hist3d

# Read in data

In [ ]:
filepath = '/home/asberk/data/4-Vadeboncoeur/davis-bay.txt'
dl = DataLoader(filepath=filepath)

In [ ]:
dl.readData()

In [ ]:
lonlat = dl.getLonLatPairs()

In [ ]:
lon_unique, lat_unique = dl.uniqueLonLat()

In [ ]:
lonlat_unique = dl.uniqueLonLatPairs()

In [ ]:
# A little show of sparsity:
point_sparsity = lonlat_unique.shape[0] / (lat_unique.size * lon_unique.size)
print('density of recorded unique pairs is only {}.'.format(np.round(point_sparsity, 3)))

In [ ]:
hist3d(dl.getScaledRgbArray(0), azim=-105)

In [ ]:
th_lon = smallestDiff(lon_unique, random=False)
th_lat = smallestDiff(lat_unique, random=False)
th_rad = np.linalg.norm([th_lon, th_lat])
print((th_lon, th_lat, th_rad))
# Expecting (9.9999999747524271e-07, 9.9999999747524271e-07, 1.4142135588025491e-06) for this

# define neighbourhood radius
rho = 3
th_nbr = rho * th_rad
print('neighbourhood radius: {}'.format(th_nbr))

In [ ]:
near_equal(lonlat[0,0], lonlat[0,1], lon_unique, lat_unique, 1e-5)

Next up: 
* define a method that gets the colour data for each of these points
* define a method that will do this for the first N points of lonlat
* turn all of that into a generator that will work with mbKMeans partial_fit

In [ ]:
def getNeighbours(lonlat, lon_vec, lat_vec, th=None):
    if (isinstance(lonlat, tuple) or isinstance(lonlat, list)) and (len(lonlat) == 2):
        # assume lonlat is a tuple of lists
        return {(lon, lat): near_equal(lon, lat, lon_vec, lat_vec, th) 
                for lon in lonlat[0] for lat in lonlat[1]}
    elif isinstance(lonlat, np.ndarray):
        return {(lon, lat): near_equal(lon, lat, lon_vec, lat_vec, th) 
                for lon,lat in lonlat}
    else:
        raise ValueError('Unsure how to parse unknown type lonlat')

In [ ]:
viridis_ = plt.cm.viridis(np.arange(300))[::30]

In [ ]:
for j, (k,v) in enumerate(getNeighbours(lonlat[:10,:], lon_unique, lat_unique, th_nbr).items()):
    plt.plot(v[:,0], v[:,1], color=viridis_[j])

In [ ]:
asdf = {1: np.array([1,2,3]), 2: np.array([1,2,3]), 4: np.array([1,2,3])}

In [ ]:
np.vstack(asdf.values())

In [ ]:
def getColour(lon, lat):
    return colour

# Converting colours

In [ ]:
from skimage import color

In [ ]:
color.rgb2lab(np.random.rand(2, 1, 3))

MiniBatch KMeans

In [ ]:
from sklearn.cluster import MiniBatchKMeans as mbKMeans

In [ ]:
mbkm = mbKMeans(n_clusters=10, verbose=True)

In [ ]:
for k in sorted(data.keys()):
    mbkm.partial_fit(data[k].loc[:, ['r', 'g', 'b']].values)

Plot resulting centers

In [ ]:
y = [0 for _ in range(10)]
x = range(10)

plt.scatter(x, y, c=mbkm.cluster_centers_/255, s=1000)